**<font color = black size=6>实验三:逻辑回归</font>**

**<font color = blue size=4>第一部分:函数介绍</font>**

<span style="color:purple">由于机器学习所使用的数据集并不一定能够直接进行使用，如经常可能出现缺失值或者异常值（如异常0或者异常大的数据），又或者是数据类型不适合直接应用于模型训练等，所以数据预处理同样是机器学习过程中十分重要的一个部分。  
    本次实验的数据预处理需要用的操作包括判断数据是否有缺失值以及异常值、缺失值替换操作、并将部分特征值类型转换为适合模型训练的类型。在进行逻辑回归实验之前，首先介绍下上述操作所用到的python语法。</span>

<span style="color:purple">1) 判断一列中是否有缺失值</span>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import matplotlib as mpl
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#通过pandas将csv文件转换为dataframe格式后进行操作
train1_frame = pd.read_csv('introduction.csv')
#判断每一列是否有空值并返回每列空值的数量
print(train1_frame.isnull().sum())
print(train1_frame)

<span style="color:purple">2) 对每一列的缺失值和异常值进行替换操作</span>

In [ ]:
# 将0替换为空值，之后将空值替换为修正的值(例如：平均值、中数、众数等)
# 利用pandas中的replace函数将某一列的指定值替换为另一个值
train1_frame[['height','weight']]=\
train1_frame[['height','weight']].replace(0,np.NaN)
print(train1_frame)

In [ ]:
#先对height列进行分离，之后将temp按照性别进行分类，之后使用mean函数分别计算male和female非空数据的height的平均值
temp=train1_frame[train1_frame['height'].notnull()]
temp=temp[['height','sex']].groupby(['sex'])[['height']].mean().reset_index()

#使用dataframe的loc函数将指定条件的height列和sex列筛选出来进行值替换操作
train1_frame.loc[(train1_frame['sex']=='Male')&(train1_frame['height'].isnull()),'height']=temp['height'][0]
train1_frame.loc[(train1_frame['sex']=='Female')&(train1_frame['height'].isnull()),'height']=temp['height'][1]
print(train1_frame)

In [ ]:
#再对weight列进行分离，之后同样将temp按照性别进行分类，之后使用median函数分别计算male和female非空数据的height的平均值
temp=train1_frame[train1_frame['weight'].notnull()]
temp=temp[['weight','sex']].groupby(['sex'])[['weight']].median().reset_index()

#使用dataframe的loc函数将指定条件的height列和sex列筛选出来进行值替换操作
train1_frame.loc[(train1_frame['sex']=='Male')&(train1_frame['weight'].isnull()),'weight']=temp['weight'][0]
train1_frame.loc[(train1_frame['sex']=='Female')&(train1_frame['weight'].isnull()),'weight']=temp['weight'][1]
print(train1_frame)

<span style="color:purple">3) 将结果值从字符串变成容易操作的整数类型</span>

In [ ]:
print(train1_frame)
train1_frame['sex'] = np.where(train1_frame['sex'] == "Male", 0, 1)
print(train1_frame)

<span style="color:purple">4) 当坐标为两维时的绘图</span>

In [ ]:
#确定图画边界和大小
plt.figure(figsize=(10,5))
x_min, x_max = 0,10
y_min, y_max = 0,5
#使用numpy中的meshgrid生成网格矩阵，方便进行之后的描点
boundary_x, boundary_y = np.meshgrid(np.arange(x_min, x_max, 0.01),np.arange(y_min, y_max, 0.01))
grid = np.c_[boundary_x.ravel(), boundary_y.ravel()]
#加入偏置对应的一列
e=np.ones((len(grid),1))
grid=np.c_[e,grid]
#假定下列的模型参数
w=np.array([[0],[3],[-6]])
#计算出网格点中每个点对应的逻辑回归预测值
z=grid.dot(w)
for i in range(len(z)):
    z[i][0]=(1/(1+np.exp(-z[i])))
    if(z[i][0]<0.5):z[i][0]=0
    else:z[i][0]=1
#转换shape以作出决策边界
z=z.reshape(boundary_x.shape)
plt.contourf(boundary_x, boundary_y, z, cmap=plt.cm.Spectral, zorder=1)
plt.show()

In [ ]:
"""
可使用plt.scatter来绘制出测试集的每个样本点，并设置指定颜色来区分预测正确和错误的样本
plt.scatter(x,y,c="color")，x、y为坐标值，c为指定颜色
"""
class_1=train1_frame[train1_frame['sex']==1]
class_0=train1_frame[train1_frame['sex']==0]
plt.scatter(class_1['height'],class_1['weight'],c='blue')
plt.scatter(class_0['height'],class_0['weight'],c='red')
plt.show()

**<font color = blue size=4>第二部分:逻辑回归实验一</font>**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import matplotlib as mpl

<span style="color:purple">Iris数据集是常用的分类实验数据集,也称鸢尾花卉数据集，是一类多重变量分析的数据集。我们实验选取数据集的部分内容，包含训练集中的80个数据样本和测试集的20个样本，分为2类，每个数据包含2个属性。可通过花萼长度(x1)，花萼宽度(x2)2个属性预测鸢尾花卉属于（Setosa，Versicolour）二个种类中的哪一类。</span>

<span style="color:purple">1) 使用pandas库将训练数据集'flower_train.csv'与测试数据集'flower_test.csv'载入到Dataframe对象中，并判断训练集中每列数据是否有缺失值或者不合理的数值，如果有，请在不删除数据的前提下进行处理，而测试集为完好的数据集，不需要进行操作。由于花卉类型(type)为字符串类型，请将花卉类型转换为适合模型训练的类型</span>

In [ ]:
train1_frame = pd.read_csv('flower_train.csv')
test1_frame = pd.read_csv('flower_test.csv')

train1_frame['type'] = np.where(train1_frame['type'] == "Iris-setosa", 0, 1)
test1_frame['type'] = np.where(test1_frame['type'] == "Iris-setosa", 0, 1)

print(train1_frame.isnull().sum())

train1_frame[['x1','x2']]=\
train1_frame[['x1','x2']].replace(0,np.NaN)

temp=train1_frame[train1_frame['x1'].notnull()]
temp=temp[['x1','type']].groupby(['type'])[['x1']].mean().reset_index()
train1_frame.loc[(train1_frame['type']==0)&(train1_frame['x1'].isnull()),'x1']=temp['x1'][0]
train1_frame.loc[(train1_frame['type']==1)&(train1_frame['x1'].isnull()),'x1']=temp['x1'][1]

temp1=train1_frame[train1_frame['x2'].notnull()]
temp1=temp1[['x2','type']].groupby(['type'])[['x2']].mean().reset_index()
train1_frame.loc[(train1_frame['type']==0)&(train1_frame['x2'].isnull()),'x2']=temp1['x2'][0]
train1_frame.loc[(train1_frame['type']==1)&(train1_frame['x2'].isnull()),'x2']=temp1['x2'][1]

print(train1_frame.isnull().sum())

train1 = np.array(train1_frame)
test1=np.array(test1_frame)

train1_feature=train1[:,0:2]
train1_outcome=train1[:,2:3]
test1_feature=test1[:,0:2]
test1_outcome=test1[:,2:3]

<span style="color:purple">2)在之前的线性回归实验中，我们的模型为$\hat{y}=\omega^T x+b$，为方便实验，该实验中我们将偏置量b划入模型参数中，则对应的模型变为$\hat{y}=\omega^Tx$，请进行相应的转换  
tips:上一次实验中的矩阵求解析解的方法中将某一列全设置为1，即将偏置量b算入模型参数中，特征值中加入一列全1的特征量</span>

In [ ]:
f1=np.ones((len(train1),1))
f2=np.ones((len(test1),1))

train1_feature=np.concatenate((f1,train1_feature),axis=1)
test1_feature=np.concatenate((f2,test1_feature),axis=1)

<span style="color:purple">3) 由于逻辑回归的原理是用逻辑函数把线性回归的结果(-∞,∞)映射到(0,1)所以逻辑回归适合于二分类问题。我们使用sigmoid函数$g(z)=\frac{1}{1+e^{-z}}$将把线性回归的结果从(-∞,∞)映射到(0,1)  
    假设模型为线性回归模型$\hat{y}=\omega_0+\omega_1 x_1+\omega_2 x_2+...+\omega_n x_n=\omega^T x$，则任意样本所对应发生的概率值函数即为$g(\hat{y})=\frac{1}{1+e^{-\hat{y}}}$，这样事情发生(定义为标签为1)的概率为$$P(y=1|x)=\frac{1}{1+e^{-\omega^T x}}$$  
    对应于任意一个样本(${x_i}$,$y_i$),其中$x_i$为特征值，$y_i$为实际结果值,在参数$\omega$下，该样本发生的概率为$$P(y_i|x_i,\omega)=y_i{P(y_i=1|x_i)}+({1-y_i}){P(y_i=0|x_i)}$$
    将每个样本发生概率相乘，得到似然函数:$$\Pi^m_{i=1}{P(y_i|x_i,\omega)}$$
    为了计算方便，一般取对数得到对数似然函数:$$L(\omega)=\sum^m_{i=1}{lnP(y_i|x_i,\omega)}$$  
    我们总是希望出现预测正确的概率的可能性最大，即想要得到极大化似然函数对应的参数$\omega$。这样最大化似然函数就转变为最小化似然函数的负数，取负的平均对数似然函数为损失函数,通过这样构建的损失函数$$J(\omega)=-\frac{1}{m}\sum^m_{i=1}{lnP(y_i|x_i,\omega)}=-\frac{1}{m}\sum^m_{i=1}ln(y_i\frac{1}{1+e^{-\omega^T x_i}}+(1-y_i)\frac{e^{-\omega^T x_i}}{1+e^{-\omega^T x_i}})$$  
    手动实现梯度下降法(不使用机器学习框架，如PyTorch、TensorFlow等)来进行模型的训练。  </span>

<span style="color:purple">算法步骤如下：①初始化模型参数$\omega$的值；②在负梯度的方向上更新参数(由于该实验涉及样本数量较小，建议使用批量梯度下降)，并不断迭代这一步骤。</span>

<span style="color:purple">其中梯度的下降偏导公式为
    $$\frac{\partial J}{\partial \omega_j}=\frac{1}{m}\sum_{i=1}^m x_{ij}(\frac{e^{\omega^T x_i}}{1+e^{\omega^T x_i}}-y_i)$$  
    参数更新的公式为$$\omega_j =\omega_j-\eta\frac{\partial J}{\partial w_j}$$其中$\eta$表示学习率，$m$则表示批量中的样本数量，$x_{ij}$代表着第i个样本的第j个特征值,$y_i$代表着第i个样本的真实值</span>

In [1]:
eta = 0.015
w=np.array([[1],[1],[1]])
for i in range(0,10000):
    a=train1_feature
    y=train1_feature
    x=y.dot(w)
    for i in range(80):
        x[i]=(np.exp(x[i][0])/(1+np.exp(x[i][0])))
    w=w-a.T.dot(x-train1_outcome)*(eta/80)
print(w)

NameError: name 'np' is not defined

<span style="color:purple">4)在模型训练完成后得到所训练的模型参数$\omega$，在测试集上进行所训练模型的测试并使用之前所介绍的损失函数计算loss值</span>

In [ ]:
outcome=test1_feature.dot(w)
for i in range(20):
    outcome[i]=math.log(1/(1+np.exp(-outcome[i])))
loss=-(np.sum(np.multiply(test1_outcome,outcome)+np.multiply((np.ones((20,1))-test1_outcome),outcome))/20)
print(loss)

<span style="color:purple">5)使用训练后的逻辑回归模型对测试数据集'flower_test.csv'进行预测，输出可视化结果（比如用seaborn或者matplotlib等可视化库来画出测试数据的散点图以及训练好的模型函数图像)，要求如下:  
    1.将所得到的逻辑回归模型所得到的决策边界绘制出来  
  2.测试集的所有点在同一幅图中进行绘制  
  3.需要给不同类别的测试点不同颜色，方便通过颜色的区别直观看到预测正确和错误的样本</span>

In [ ]:
plt.figure(figsize=(9,6))
x_min, x_max = 4.2,7.2
y_min, y_max = 2.2,4.2
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),np.arange(y_min, y_max, 0.01))
d = np.c_[xx.ravel(), yy.ravel()]
e=np.ones((len(d),1))
d=np.c_[e,d]
z=d.dot(w)
for i in range(len(z)):
    z[i][0]=(1/(1+np.exp(-z[i])))
    if(z[i][0]<0.5):z[i][0]=0
    else:
        z[i][0]=1
z=z.reshape(xx.shape)
plt.contourf(xx, yy, z, cmap=plt.cm.Spectral, zorder=1)
class_1=test1_frame[test1_frame['type']==1]
class_0=test1_frame[test1_frame['type']==0]
plt.scatter(class_1['x1'],class_1['x2'],c='blue')
plt.scatter(class_0['x1'],class_0['x2'],c='red')
plt.show()


**<font color = blue size=4>第三部分:逻辑回归实验二</font>**

<span style="color:purple">该数据集(train_titanic.csv和test_titanic.csv)同样为分类数据集，为泰坦尼克号的乘客信息以及最后是否生还。包括了七个特征值以及一个类别特征(即为Survived类型,代表是否生还),特征信息分别为Passengerid(乘客id)，Age(乘客年龄)，Fare(船票价格),Sex(性别)，sibsp(堂兄弟妹个数)，Parch(父母与小孩的个数)，Pclass(乘客等级)</span>

<span style="color:purple">该数据集已经做了处理，无缺失值和空值，且字符串类型全部转换成了整数类型，你们需要进行判断，在七个特征值至少选择四个你认为与最后是否生还关联度高的特征类别。该实验的任务依然是在训练集上使用逻辑回归方法和手动实现的梯度下降方法完成模型训练。</span>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

<span style="color:purple">1) 使用pandas库将训练数据集'train_titanic.csv'与测试数据集'test_titanic.csv'载入到Dataframe对象中</span>

In [ ]:
train2_frame = pd.read_csv('train_titanic.csv')
test2_frame = pd.read_csv('test_titanic.csv')

train2_feature=train2_frame[['Age','Fare','Sex','Pclass']]
train2_outcome=train2_frame[['Survived']]
test2_feature=test2_frame[['Age','Fare','Sex','Pclass']]
test2_outcome=test2_frame[['Survived']]
print(train2_feature.shape)
train2_f = np.array(train2_feature)
train2_o = np.array(train2_outcome)
test2_f  = np.array(test2_feature)
test2_o  = np.array(test2_outcome)

f1=np.ones((len(train2_f),1))
f2=np.ones((len(test2_f),1))

train2_f = np.concatenate((f1,train2_f),axis=1)
test2_f  = np.concatenate((f2,test2_f),axis=1)
print(train2_f)

<span style="color:purple">2)利用上个实验所使用的梯度下降方法(由于该数据集样本数量较大，所以建议使用随机批量或小批量)进行模型训练</span>

In [ ]:
eta = 0.0015
w=np.array([[1],[1],[1],[1],[1]])
for i in range(0,10000):
    a=train2_f
    y=train2_f
    x=y.dot(w)
    for i in range(1009):
        x[i]=(np.exp(x[i][0])/(1+np.exp(x[i][0])))
    w=w-a.T.dot(x-train2_o)*(eta/1009)
print(w)

<span style="color:purple">3) 使用训练后的逻辑回归模型对测试数据集'test_titanic.csv'进行预测，并计算其loss值：</span>

In [ ]:
outcome=test2_f.dot(w)
for i in range(300):
    outcome[i]=math.log(1/(1+np.exp(-outcome[i])))
loss=-(np.sum(np.multiply(test2_o,outcome)+np.multiply((np.ones((300,1))-test2_o),outcome))/300)
print(loss)